<a href="https://colab.research.google.com/github/itssofiarce/NLP-2025/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Procesamiento del Lenguaje Natural - 2025

### Trabajo Práctico

Arce, Sofía.

## Preparación de entorno

In [1]:
# Clonamos el reposotorio y el directorio de los datos
import os

REPO_NAME = "NLP-2025"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/itssofiarce/{REPO_NAME}.git
  os.chdir(REPO_NAME)

Cloning into 'NLP-2025'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 115 (delta 21), reused 71 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 12.05 MiB | 11.17 MiB/s, done.
Resolving deltas: 100% (21/21), done.


### Pre analísis de la información .txt

In [2]:
# Texto mas extenso
longitud = {}
informacion = os.listdir('/content/NLP-2025/NLP_CASCADIA/datos/información')

for texto in informacion:
  contenido = open(f'/content/NLP-2025/NLP_CASCADIA/datos/información/{texto}', 'r')
  # Cuento solo las palabras

  longitud[texto] = len(contenido.read().split())

print(longitud)


{'whatboardgame_review.txt': 2288, 'oneboardfamily_review.txt': 1592, 'flatout_games.txt': 298, 'cascadia_manual.txt': 6734, 'board_game_co_uk_guide.txt': 127, 'bluehighwaygames_description.txt': 908, 'boardgamereview_review.txt': 2606}


En base al output de arriba el texto mas extenso es el manual de cascadia con 6734 palabras.



Para facilitar el desarrollo del Trabajo Práctico crearé la clase "TextoParaAnalizar" con los metodós necesarios para completar los enunciados.

In [3]:
import pandas as pd

In [29]:
# Funciones adicionales
def entorno_modelo(tipo_embedding):
  if tipo_embedding == "CountVectorizer":

    from sklearn.feature_extraction.text import CountVectorizer
    model = CountVectorizer()

  elif tipo_embedding == "sentencetransformer":
    !pip install -U sentence-transformers
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('all-MiniLM-L6-v2')

  elif tipo_embedding == "spacy":
    import spacy
    model = spacy.load('en_core_web_sm')

  return model


In [115]:
class TextoParaAnalizar:

  def __init__(self, text):
    # Validar path del archivo
    if not os.path.exists(text):
      raise Exception('El archivo no existe')
    self.text = text

  def __str__(self):
    """ Devuelve el contenido del texto """
    with open(self.text, 'r') as f:
      return f.read()

  def fragmentos(self, fragmento="palabra"):
    """ Devuelve una lista de fragmentos del texto.
      - "Fragmento: oracion"
      - "Fragmento: palabra"
    """
    import re
    texto_limpio = re.sub(r'[().!]', '', self.__str__())
    if fragmento == "oracion":
      texto_limpio =[line for line in texto_limpio.splitlines() if line.strip()]
      return texto_limpio
    return texto_limpio.split()


  def vectorizar(self, tipo: str = "sentencetransformer"):
    """ Vectoriza los fragmentos y los devuelve en un dataset. Por default usa ul modelo ContextDependent como word2vec.
        tipo:
        - "ContextDependent: CountVectorizer"
        - "ContextIndependent: "" sentencetransformer """
    import pandas as pd
    modelo = entorno_modelo(tipo)
    oraciones = self.fragmentos("oracion")
    embeddings = []

    if tipo == "CountVectorizer":
      X = modelo.fit_transform(oraciones)
      response_new = pd.DataFrame(X.toarray(), columns=modelo.get_feature_names_out())

    elif tipo == "sentencetransformer":
      response_new = modelo.encode(oraciones)

    return response_new, modelo

  def extraer_sustantivos(self):
      """ Extrae los sustantivos de un texto. Part of Speech Tagging """


      modelo = entorno_modelo("spacy")
      oraciones = self.fragmentos("oracion")

      sustantivos_en_oraciones = []
      entidades_con_ner = {}
      sustantivos_en_oraciones_categorizados = {}

      for oracion in oraciones:
        doc = modelo(oracion)

        # NER
        for ent in doc.ents:
          if ent.text.lower() not in entidades_con_ner.keys():
            entidades_con_ner[ent.text.lower()] = ent.label_

        # POS
        for token in doc:
          if token.pos_ == "NOUN":
            sustantivo = token.text.lower()
            if sustantivo not in sustantivos_en_oraciones:
              sustantivos_en_oraciones.append(sustantivo)

        # Agreo al diccionario aquellos sustantivos que no fueron identificados como ner
        for sustantivo in sustantivos_en_oraciones:
          if sustantivo not in entidades_con_ner.keys():
            sustantivos_en_oraciones_categorizados[sustantivo] = ''

        # Junto ambos diccionarios
        sustantivos_en_oraciones_categorizados.update(entidades_con_ner)

      return sustantivos_en_oraciones_categorizados

## EJERCICIO 2
Apoyándose en la sección de información. Separa en fragmentos un texto extenso extraído y vectoriza cada fragmento con alguno de los modelos de embedding vistos en clases.

In [26]:
texto_extenso = TextoParaAnalizar('/content/NLP-2025/NLP_CASCADIA/datos/información/cascadia_manual.txt')

En el bloque de codigo de arriba, dataframe es un diccionario que almacena en forma de lista, las palabras de las oraciones, esto el permite a word2vec aprender mejor el contexto de las mismas. Luego entrenamos el modelo y lo guardamos.

Las palabras "rules" y "games" tienen una similitud cerca de uno por lo que significa que son palabras cuyos embeddings estan cerca en el espacio y por lo tanto, estan relacionadas en cuanto a significado. Ahora entrenaré otro modelo para probarlo con frases.

In [7]:
# Separo en palabras y vectorizo
dataframe, model = texto_extenso.vectorizar("CountVectorizer")

In [8]:
# Separo en frases
dataframe, model = texto_extenso.vectorizar("sentencetransformer")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Luego realiza un análisis de similitud de texto ingresando varias frases a buscar semánticamente, compare distintas técnicas de distancias vistas en clases, elija la mejor y justifique la razón por la que esa técnica se ajusta para este tipo de búsquedas.



In [9]:
# Calcular la similutid del coseno entre las frases_ejemplo y todo el texto.
from sentence_transformers import util
frases_ejemplo=["Casadia is a game", "There are only 3 types of Wildlife cards", "Casadia ia a geographic region"]
frases_ejemplo_vectorizadas = model.encode(frases_ejemplo)

In [10]:
to_plot_coincidencias=[]
to_plot_coincidencias_vectorizadas=[]

In [11]:
for i, frase in enumerate(frases_ejemplo):
    similitudes = util.cos_sim(frases_ejemplo_vectorizadas[i], dataframe)
    print(f"\nFrase ejemplo: '{frase}'")
    # Mostrar las frases del texto más similares a esta
    resultados_ordenados = similitudes[0].argsort(descending=True)
    for idx in resultados_ordenados[:3]:  # Top 3 resultados
        # Busco por idx la oracion original
        to_plot_coincidencias.append(texto_extenso.fragmentos("oracion")[idx])
        to_plot_coincidencias_vectorizadas.append(dataframe[idx])
        oracion_similar = texto_extenso.fragmentos("oracion")[idx.item()]
        print(f" → {oracion_similar} (Similitud: {similitudes[0][idx]:.4f})")


Frase ejemplo: 'Casadia is a game'
 → Set up and play a multi-player game of Cascadia using the  (Similitud: 0.5859)
 → Set up and play a multi-player game of Cascadia following  (Similitud: 0.5751)
 → The Flatout Games CoLab for Cascadia is: (Similitud: 0.5627)

Frase ejemplo: 'There are only 3 types of Wildlife cards'
 → Only have 3 types of Wildlife (Similitud: 0.7758)
 → 1 Wildlife Scoring Cards (Similitud: 0.7268)
 → 1 Wildlife Scoring Cards (Similitud: 0.7268)

Frase ejemplo: 'Casadia ia a geographic region'
 → About the Region (Similitud: 0.5014)
 → landmarks in the region (Similitud: 0.4970)
 → wild places wherever you may be, or consider paying Cascadia a visit (Similitud: 0.4175)


In [12]:
# Similitud de Jaccard
def jaccard_similarity(str1, str2):
    set1 = set(str1.lower().split())
    set2 = set(str2.lower().split())
    return len(set1 & set2) / len(set1 | set2) if set1 | set2 else 0

In [15]:
from sentence_transformers import util

frases_ejemplo = ["Casadia is a game", "There are only 3 types of Wildlife cards", "Casadia ia a geographic region"]

for i, frase in enumerate(frases_ejemplo):
    print(f"\nFrase ejemplo: '{frase}'")

    # Calculate Jaccard similarity for each fragment in the text
    similitudes_jaccard = [jaccard_similarity(frase, oracion) for oracion in texto_extenso.fragmentos("oracion")]

    # Get the indices of the top 3 most similar sentences
    resultados_ordenados_jaccard = sorted(range(len(similitudes_jaccard)), key=lambda i: similitudes_jaccard[i], reverse=True)[:3]

    # Print the top 3 most similar sentences and their Jaccard similarities
    for idx in resultados_ordenados_jaccard:
        oracion_similar = texto_extenso.fragmentos("oracion")[idx]
        print(f" → {oracion_similar} (Similitud Jaccard: {similitudes_jaccard[idx]:.4f})")


Frase ejemplo: 'Casadia is a game'
 → This tile is a Keystone  (Similitud Jaccard: 0.2857)
 → A (Similitud Jaccard: 0.2500)
 → A (Similitud Jaccard: 0.2500)

Frase ejemplo: 'There are only 3 types of Wildlife cards'
 → Only have 3 types of Wildlife (Similitud Jaccard: 0.5556)
 → If 3 of the available Wildlife Tokens are the same, then  (Similitud Jaccard: 0.2857)
 → Overpopulation: 3 of the same Wildlife Token (Similitud Jaccard: 0.2500)

Frase ejemplo: 'Casadia ia a geographic region'
 → A (Similitud Jaccard: 0.2000)
 → A (Similitud Jaccard: 0.2000)
 → A (Similitud Jaccard: 0.2000)


OPCIONAL: Visualizar en 3D aplicando PCA o t-SNE la ubicación de los fragmentos y la query ingresada vectorizada en el espacio. Realizar una observación sobre la visualización.


In [16]:
!pip install plotly

In [17]:
import plotly.graph_objects as go
from sklearn.decomposition import PCA
import numpy as np

# Initialize lists to store all data points for PCA
all_vectores = []
all_oraciones = []
all_colores = []

for i, frase in enumerate(frases_ejemplo):
    similitudes = util.cos_sim(frases_ejemplo_vectorizadas[i], dataframe)
    print(f"\nFrase ejemplo: '{frase}'")

    resultados_ordenados = similitudes[0].argsort(descending=True)

    # Store data for current phrase
    all_oraciones.append(frase)  # include the example phrase
    all_vectores.append(frases_ejemplo_vectorizadas[i])
    all_colores.append('red')

    for idx in resultados_ordenados[:3]:
        oracion_similar = texto_extenso.fragmentos("oracion")[idx.item()]
        print(f" → {oracion_similar} (Similitud: {similitudes[0][idx]:.4f})")

        # Store data for similar sentences
        all_oraciones.append(oracion_similar)
        all_vectores.append(dataframe[idx.item()])
        all_colores.append('blue')


# Apply PCA to all collected data points
pca = PCA(n_components=3)
vectores_3d = pca.fit_transform(np.array(all_vectores))
x, y, z = vectores_3d[:, 0], vectores_3d[:, 1], vectores_3d[:, 2]
hover_text = [f"{j}: {sent}" for j, sent in enumerate(all_oraciones)]

# Create a single plot with all data points
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers+text',
    marker=dict(
        size=6,
        color=all_colores,
        opacity=0.8
    ),
    text=[str(j) for j in range(len(all_oraciones))],
    hovertext=hover_text,
    hoverinfo='text'
))

fig.update_layout(
    title='Visualización de Frases Ejemplo y Similares',
    scene=dict(
        xaxis_title='PC 1',
        yaxis_title='PC 2',
        zaxis_title='PC 3'
    ),
    showlegend=False
)

fig.show()


Frase ejemplo: 'Casadia is a game'
 → Set up and play a multi-player game of Cascadia using the  (Similitud: 0.5859)
 → Set up and play a multi-player game of Cascadia following  (Similitud: 0.5751)
 → The Flatout Games CoLab for Cascadia is: (Similitud: 0.5627)

Frase ejemplo: 'There are only 3 types of Wildlife cards'
 → Only have 3 types of Wildlife (Similitud: 0.7758)
 → 1 Wildlife Scoring Cards (Similitud: 0.7268)
 → 1 Wildlife Scoring Cards (Similitud: 0.7268)

Frase ejemplo: 'Casadia ia a geographic region'
 → About the Region (Similitud: 0.5014)
 → landmarks in the region (Similitud: 0.4970)
 → wild places wherever you may be, or consider paying Cascadia a visit (Similitud: 0.4175)


de la teoria:: Los métodos Count Vectorizer y TF-IDF capturan la aparición o frecuencia de palabras. Si bien admiten el uso
de similitud de coseno, la cercanía estará dada por las similitudes en cuanto a la presencia o cuenta de las
mismas palabras, y no por su similitud semántica o de contexto, como en el caso de los modelos de
embeddings.


## EJERCICIO 3
Apoyándose nuevamente en la sección de información. Recoge un texto extenso extraído, divídelos en fragmentos, luego realiza extracciones de sustantivos (POS) y categoriza estos sustantivos (NER), a continuación realiza una búsqueda de similitud filtrando por sustantivos, compara las distintas técnicas de distancias vistas en clases, elija la mejor y justifique la razón por la que esta técnica se ajusta para este tipo de búsquedas.


{
  "sustantivo_nombre": [[lista de ent que contienen el sustantivo_nombre], ent]
}

In [116]:
texto_extenso_2 = TextoParaAnalizar('/content/NLP-2025/NLP_CASCADIA/datos/información/oneboardfamily_review.txt')

In [119]:
sustantivos_clasificados = texto_extenso_2.extraer_sustantivos()

In [128]:
# Busqueda de similitud
sutantivos_a_buscar = ['game', '']

In [126]:
similitudes

tensor([[ 1.0000,  0.0777,  0.1254,  ..., -0.0136,  0.1791,  0.0621],
        [ 0.0777,  1.0000,  0.2454,  ...,  0.3103,  0.1492,  0.2354],
        [ 0.1254,  0.2454,  1.0000,  ...,  0.3066,  0.2625,  0.2873],
        ...,
        [-0.0136,  0.3103,  0.3066,  ...,  1.0000,  0.1786,  0.2758],
        [ 0.1791,  0.1492,  0.2625,  ...,  0.1786,  1.0000,  0.1965],
        [ 0.0621,  0.2354,  0.2873,  ...,  0.2758,  0.1965,  1.0000]])

## EJERCICIO 4
Mediante detección de idioma, separar los archivos en distintos lenguajes y guardar esa
información en un dataframe.


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=c38ec347fbd338f38ccd737a88ae04a78a0056c669c31aacf90ea16c0a692c50
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
import os
import pandas as pd
from langdetect import detect, LangDetectException

def detect_language(text_path):
    try:
        with open(text_path, 'r', encoding='utf-8') as file:
            text = file.read()
            return detect(text)
    except LangDetectException:
        return "unknown"  # En caso de que falle al detectar el idioma
    except Exception as e:
        print(f"Error processing {text_path}: {e}")
        return "error"


# Directorio que contiene los archivos de texto
directorio_informacion = '/content/NLP-2025/NLP_CASCADIA/datos/información'

# Lista para almacenar la información de los archivos
datos_archivos = []

for filename in os.listdir(directorio_informacion):
    if filename.endswith(".txt"):
        ruta_completa = os.path.join(directorio_informacion, filename)
        idioma = detect_language(ruta_completa)
        datos_archivos.append([filename, idioma])


# Crear un DataFrame de pandas
df_idiomas = pd.DataFrame(datos_archivos, columns=["Nombre del archivo", "Idioma"])

df_idiomas


,Nombre del archivo,Idioma
0,whatboardgame_review.txt,en
1,board_game_co_uk_guide.txt,en
2,flatout_games.txt,en
3,boardgamereview_review.txt,en
4,cascadia_manual.txt,en
5,bluehighwaygames_description.txt,en
6,oneboardfamily_review.txt,en


##EJERCICIO 5
En el caso de las reseñas realizadas por usuarios, utiliza análisis de sentimientos con modelos pre entrenados y guarda la clasificación predecida de cada reseña.

Luego, crea un sistema de búsquedas por similitud semántica y que permita filtrar por sentimiento para obtener.


## EJERCICIO 6
Crea un set de datos de consultas (más de 300 preguntas en total) y categorizalas entre la
fuente de datos que pueda llegar a responder esa pregunta entre estadísticas,
información y relaciones.
Por ejemplo:
- ¿Cómo gano en el ajedrez? -> Información
- ¿Quién trabajó para el ta-te-ti? -> Relaciones
- ¿Qué puntaje tienen las damas? -> Estadística

A continuación, transforma esas consultas en vectores y entrena un modelo de clasificación
(a gusto del estudiante) en donde pueda predecir la categoría a través de la consulta
ingresada.
Agregar métricas y análisis durante todo el desarrollo, trabaje en varios modelos y
comparelos.